### Developing backtesting strategy

We are going to develop a trading strategy around the predictions from our logistic regression model. The general flow goes like this: 
* Long-only strategy - we will only buy ETH/USD, we will not short 
* Our entry is when we do not have a position and we get a signal of +1 
    * We say +1 if the probability of +1 is higher than some $p_{up}$
* Our exit from the long position is triggered if one of these two conditions happens: 
    * We hold the position for longer than $N$ periods
    * We get a signal of -1 which means that the probability of -1 if higher than some $p_{down}$

I think that this is a good and simple way to set things up. The parameters for the strategy are: 
* $N$ -> holding period in number of bars
* $p_{up}$ -> threshold for the +1 signal 
* $p_{down}$ -> threshold for the -1 signal

Some other details to get clear on: 
* Sizing -> we are going to do a fixed amount of 3 ETH 
* Total starting equity -> We are starting with $10,000
* Commission -> We are going to do 0.1% commission rate on all of the trades


In [20]:
import pandas as pd 
import numpy as np 
import csv
import backtrader as bt

from backtrader.feeds import PandasData
from backtrader.order import Order
from pathlib import Path

In [19]:
prepared_data = pd.read_csv("model_prepared_data.csv", index_col=0)
prepared_data.dropna(inplace = True)

prepared_data.index = pd.to_datetime(prepared_data.index)

In [54]:
class FixedCommission(bt.CommInfoBase): 

    params = {
        'commission': 0.02, 
        'stocklike': True, 
        'commtype': bt.CommInfoBase.COMM_FIXED
    }

    def _getcommission(self, size, price, pseudoexec):
        return abs(size) * self.p.commission
    
class SignalData(PandasData): 

    cols = ['open', 'high', 'low', 'close', 'volume', 'down', 'neutral', 'up']

    lines = tuple(cols)

    # define parameters
    params = {c: -1 for c in cols}
    params.update({'datetime': None})
    params = tuple(params.items())

class LogisticRegStrategy(bt.Strategy): 

    params = {
        'holding_period': 48, 
        'verbose': False, 
        'log_file': 'backtest.csv', 
        'p_up_threshold': 0.4, 
        'p_down_threshold': 0.4
    }

    def log(self, txt, dt = None): 

        dt = dt or self.datas[0].datetime.datetime(0)

        with Path(self.p.log_file).open('a') as f: 
            log_writer = csv.writer(f)
            log_writer.writerow([dt.isoformat()] + txt.split(","))
        
        print(dt, txt)

    def __init__(self): 

        self.dataclose = self.datas[0].close
        self.up_prob = self.datas[0].up 
        self.down_prob = self.datas[0].down 
        self.neutral_prob = self.datas[0].neutral

        # Keeping track of orders adn stuff 
        self.order = None 
        self.buyprice = None 
        self.buycomm = None

    def notify_order(self, order: Order): 

        if order.status in [Order.Submitted, Order.Accepted]:
            return 

        if order.status in [Order.Completed]: 
            p = order.executed.price
            if order.isbuy(): 
                self.log(f'{order.data._name}, BUY executed, {p:.2f}')
            elif order.issell(): 
                self.log(f'{order.data._name}, SELL executed, {p:.2f}') 
            
            self.bar_executed_at = len(self) # Storing when this happened

        elif order.status in [Order.Canceled, Order.Margin, Order.Rejected]: 
            self.log(f'{order.data._name}, Order Canceled/Margin/Rejected')

        self.order = None # Restting the order to None

    def prenext(self): 
        self.next() 

    def next(self): 

        up_signal = (self.up_prob[0] > self.down_prob[0]) and (self.up_prob > self.neutral_prob[0]) and (self.up_prob[0] > self.p.p_up_threshold)
        down_signal = (self.down_prob[0] > self.up_prob[0]) and (self.down_prob[0] > self.neutral_prob[0]) and (self.down_prob[0] > self.p.p_down_threshold)

        if not self.position: 
            if up_signal: 
                self.log(f"CREATING BUY ORDER at {self.dataclose[0]:.2f}")
                self.order = self.buy()

        if self.position: 
            if down_signal or (len(self) > self.bar_executed_at + self.params.holding_period): 
                self.log(f'CREATING SELL ORDER at {self.dataclose[0]:.2f}')
                self.order = self.sell()


class LogisticRegStrategyBracket(bt.Strategy): 

    params = {
        'holding_period': 48, 
        'verbose': False, 
        'log_file': 'backtest.csv', 
        'p_up_threshold': 0.4, 
        'p_down_threshold': 0.4, 
        'take_profit_pct': 0.03, 
        'stop_loss_pct': 0.03
    }

    def log(self, txt, dt = None): 

        dt = dt or self.datas[0].datetime.datetime(0)

        with Path(self.p.log_file).open('a') as f: 
            log_writer = csv.writer(f)
            log_writer.writerow([dt.isoformat()] + txt.split(","))
        
        print(dt, txt)

    def __init__(self): 

        self.dataclose = self.datas[0].close
        self.up_prob = self.datas[0].up 
        self.down_prob = self.datas[0].down 
        self.neutral_prob = self.datas[0].neutral

        # Keeping track of orders adn stuff 
        self.main_order = None
        self.stop_loss_order = None 
        self.take_profit_order = None 
        self.buyprice = None 
        self.buycomm = None

    def notify_order(self, order: Order): 
        
        if order.status in [Order.Submitted, Order.Accepted]: 
            return 
        
        if order.status in [Order.Completed]: 
            executed_price = order.executed.price
            if order.isbuy(): 
                self.log(f'{order.data._name}, BUY executed, {executed_price:.2f}')
            elif order.issell(): 
                self.log(f'{order.data._name}, SELL executed, {executed_price:.2f}')

                # This means that a sell just happened, we can refresh everything
                self.main_order = None 
                self.take_profit_order = None 
                self.stop_loss_order = None

            self.bar_executed_at = len(self) # Storing when this happened

        elif order.status in [Order.Canceled, Order.Margin, Order.Rejected]: 
            self.log(f'{order.data._name}, Order Canceled/Margin/Rejected')


    def next(self):

        up_signal = (self.up_prob[0] > self.down_prob[0]) and (self.up_prob > self.neutral_prob[0]) and (self.up_prob[0] > self.p.p_up_threshold)
        down_signal = (self.down_prob[0] > self.up_prob[0]) and (self.down_prob[0] > self.neutral_prob[0]) and (self.down_prob[0] > self.p.p_down_threshold)

        if not self.position: 
            if up_signal: 
                # In the case of an up signal, we are going to create a bracket order with a take profit and stop loss 
                # stop_loss_price = self.dataclose[0] * (1 - self.p.stop_loss_pct)
                # take_profit_price = self.dataclose[0] * (1 + self.p.take_profit_pct)

                self.log(f'CREATING BUY ORDER at {self.dataclose[0]:.2f}')
                # self.main_order = self.buy(exectype=Order.Market, transmit=False)

                # self.log(f'CREATE STOP LOSS ORDER with price of {stop_loss_price:.2f}')
                # self.stop_loss_order = self.sell(price = stop_loss_price, 
                #                                  size = self.main_order.size, exectype=Order.Stop, 
                #                                  transmit=False, parent = self.main_order)
                # self.log(f'CREATE TAKE PROFIT ORDER with price of {take_profit_price:.2f}')
                # self.take_profit_order = self.sell(price = take_profit_price, 
                #                                    size = self.main_order.size, 
                #                                    exectype=Order.Limit, transmit=True, parent = self.main_order)

                self.main_order = self.buy()
                self.stop_loss_order = None 
                
        elif self.stop_loss_order is None: 
            self.stop_loss_order = self.sell(
                exectype=Order.StopTrail, 
                trailpercent=0.05
            )





In [55]:

c = bt.Cerebro() 
c.addstrategy(LogisticRegStrategyBracket)
c.adddata(SignalData(dataname = prepared_data))
print(c.broker.getvalue())
c.broker.setcash(10000)
c.addsizer(bt.sizers.FixedSize, stake = 1)
c.broker.setcommission(commission = 0.001)
c.run(maxcpus = 1)
print(c.broker.get_value())

10000.0


2021-01-03 13:00:00 CREATING BUY ORDER at 875.20
2021-01-03 14:00:00 , BUY executed, 874.96
2021-01-03 18:00:00 , SELL executed, 889.95
2021-01-03 18:00:00 CREATING BUY ORDER at 917.72
2021-01-03 19:00:00 , BUY executed, 917.63
2021-01-04 07:00:00 , SELL executed, 1055.50
2021-01-04 07:00:00 CREATING BUY ORDER at 1045.90
2021-01-04 08:00:00 , BUY executed, 1045.07
2021-01-04 09:00:00 , SELL executed, 970.66
2021-01-04 09:00:00 CREATING BUY ORDER at 970.00
2021-01-04 10:00:00 , BUY executed, 969.66
2021-01-04 16:00:00 , SELL executed, 992.61
2021-01-04 16:00:00 CREATING BUY ORDER at 1010.64
2021-01-04 17:00:00 , BUY executed, 1011.42
2021-01-05 03:00:00 , SELL executed, 1060.46
2021-01-05 03:00:00 CREATING BUY ORDER at 1002.97
2021-01-05 04:00:00 , BUY executed, 1002.53
2021-01-06 20:00:00 , SELL executed, 1136.20
2021-01-06 20:00:00 CREATING BUY ORDER at 1200.37
2021-01-06 21:00:00 , BUY executed, 1201.35
2021-01-07 18:00:00 , SELL executed, 1209.52
2021-01-08 03:00:00 CREATING BUY ORD

### Conclusions 

We ran our first ML strategy using this logistic regression model. There are a massive amount of things that we can tweak with this model though: 
* Do we want to have some take profit orders in there. Maybe there are times when we are gettng the general direction right but we are holding for too long and the market turns against us 
* Maybe we can try out bracker orders where we have a stop loss and a take profit. This limits our losses as well as makes sure that we lock in some gains when things go our way
* How can we adjust the parameters of probability of up and down to do well? 

As I see it, there are a couple ways that you can optimize things 
* Make the model better 
* Make the strategy parameters better
* Both